In [1]:
from math import sqrt 
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import sklearn
import pandas as pd
import numpy as np
import scipy.stats as ss
from scipy import stats
from statsmodels.regression import linear_model
import statsmodels.api as sm

In [2]:
df = pd.read_csv('data_clean.csv')

In [4]:
df.columns

Index(['State', 'ST', 'Fips', 'County', 'Precincts', 'Votes',
       'Less Than High School Diploma', 'At Least High School Diploma',
       'At Least Bachelors's Degree', 'Graduate Degree', 'School Enrollment',
       'Median Earnings 2010', 'White (Not Latino) Population',
       'African American Population', 'Native American Population',
       'Asian American Population', 'Other Race or Races', 'Latino Population',
       'Children Under 6 Living in Poverty',
       'Adults 65 and Older Living in Poverty', 'Total Population',
       'Preschool.Enrollment.Ratio.enrolled.ages.3.and.4',
       'Poverty.Rate.below.federal.poverty.threshold', 'Gini.Coefficient',
       'Child.Poverty.living.in.families.below.the.poverty.line',
       'Management.professional.and.related.occupations',
       'Service.occupations', 'Sales.and.office.occupations',
       'Farming.fishing.and.forestry.occupations',
       'Construction.extraction.maintenance.and.repair.occupations',
       'Production.tran

In [3]:
df_votes= pd.read_csv('votes_clean.csv')

In [5]:
df_votes.columns

Index(['Democrats 08 (Votes)', 'Democrats 12 (Votes)',
       'Republicans 08 (Votes)', 'Republicans 12 (Votes)', 'votes16_trumpd',
       'votes16_clintonh', 'Fips', 'Democrats08_Voteshare',
       'Republicans08_Voteshare', 'Democrats12_Voteshare',
       'Republicans12_Voteshare', 'Democrats16_Voteshare',
       'Republicans16_Voteshare'],
      dtype='object')

In [4]:
df_1 = pd.merge(df,df_votes, on = 'Fips')

In [5]:
df_test = pd.concat([df, df_votes['Republicans08_Voteshare'],df_votes['Democrats08_Voteshare'],\
                    df_votes['Republicans12_Voteshare'],df_votes['Democrats12_Voteshare']], axis=1)

In [6]:
df.drop(['State','ST','County','Fips'], axis = 1, inplace = True)
df_test.drop(['State','ST','County','Fips'], axis = 1, inplace = True)

All the dataset has been fitted to linear model as shown below.<br>
**a. What are the data points?**<br>
The data points for the regression here are the counties plotted for each independent variable against Relative vote share.   <br>
**b. What are the independent and dependent variables you’ve chosen?**<br>
The independent variables are all the columns here in dataframe `df` except the categorical column like state, county and the dependent variable is Relative Vote share from the dataframe `df_1` for the year 2008.<br>

**c. Good Measured to evaluate model fit**<br>

*R square:* After running the regression, the R square value came out to be 0.669 which means the model takes almost 67% of the variability into consideration. The higher the R square the better fit is your model. So in this case our model explains good variation of our data.   <br>

*Adjusted R square:* R square and adjusted R square are very similar except. As we add more variables R square increases however Adjusted R square increases only if the new regressor improves our model. And since we are adding too many regressors in our equation, this would be good measure here as well. In this case the adjusted R square is close to R square value and it actually drops if few independent varibales are removed which means the model we have is a better fit with these variables. <br>

*Prob(F-statistic):*The probability of F statistic is another good measure and it shows if the model is good fit than having a model wihtout independent variables or in other words if independent variables imporve the fit of the model. As seen in the summary table, Probability is very low which indicates the model is a better fit. <br>



In [7]:
def regression(x,y):
    x = x.values
    X = sm.add_constant(x)
    y = y.values
    reg = linear_model.OLS(y,X).fit()
    print(reg.summary())

In [8]:
regression(df.iloc[:,:],df_1['Republicans08_Voteshare'])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     147.6
Date:                Sun, 28 Apr 2019   Prob (F-statistic):               0.00
Time:                        19:56:21   Log-Likelihood:                 3461.5
No. Observations:                3141   AIC:                            -6835.
Df Residuals:                    3097   BIC:                            -6569.
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1139      4.125      0.270      0.7

**Interpretation:**<br>
After looking at the summary result and considering all these measures, model we have takes a good consideration of our data variance and is a good fit. However there are several things to consider here<br> 
1. We will have to find out if the model is overfit. Overfitting may arise because of adding too many independent variables and model capturing random noise of our sample rather than reflecting the general trend. Our model should not only fit the current sample, but new samples as well. To find out if our model is overfit, we will have to do train-test split and find out if our model is able to capture a genaral trend from training set and whether it can predict on test set. 
1. If we take a look at the p-values of the coefficients, only few p-vales are statistically significant and if we delete few non-significant variables from the equation, the R square values actually drops along with adjusted R sqaure. This could be becaus of multicollinearity among the variables. Not all the cariables are needed to build a model however it is not only hard to determine which variables are worth keeping but also time consuming if we keep testing each variable. Multicollinearity affects the coeffiecients and p-values but does not influence the predictions, precision of predictions and the goodness of fit statistics. Since our primary goal here is to fit the model for better predictions and not understand the role of each independent variable, we can use regularization to give importance to important parameters thus reducing the complexity of the model if needed.   

# Logistic Regression Stats model:
The following model is logistic regression model using the same demographic data to predict which political party will take the majority of a county’s votes. (I have used statsmodels package to create your logistic regression model here, not scikit-learn.) 
### Questions answered:
a) How good is the fit of your model? What metric is a good measure to evaluate your model here? Interpret your results.



In [9]:
x = df.iloc[:,:].values
X = sm.add_constant(x)
Y = df_1['Republicans08_Voteshare'].values
logit = sm.Logit(Y, X)

fitted_model = logit.fit()

prediction = fitted_model.predict(X)
print('MSE: '+str(np.mean((prediction-Y)**2)))
print('RMSE: '+str(sqrt(np.mean((prediction-Y)**2))))
print('Predicted '+str(prediction[prediction>0.5].size)+' counties vs actual '+ str(Y[Y>0.5].size))

Optimization terminated successfully.
         Current function value: 0.620330
         Iterations 6
MSE: 0.006368736603451436
RMSE: 0.07980436456392243
Predicted 2450 counties vs actual 2235


**Measure of a good fit and Interpretation:**<br>
To measure the good fit of the model either Mean Square Error or Root Mean Square error would be good metrics as they measure the variance in the predicted y values and actual. RMSE can be easily interpreted a standard deviation of the error and it has the same unit as the response variable, in our case relative vote share unit. Closer the values to zero better is the fit of our model.<br>
The model above predicts Republican party to take the majority of the votes. As shown above, model predicted 2450 counties with more than 50% Republic party vote share which is higher than the actual count 2250.<br>
The count does not indicate if our model is a better fit to the data points. MSE and RMSE values for the model are very low which indicates our model is good fit.
However the data we trained the model on acts as test data above for prediction which it is not a good to measure our model. The model needs to see new data to draw any strong conclusion about the model.

# Predictive Modelling:
I built new models to predict the 2012 and 2016 relative vote share of the Republican party, respectively.
### Questions answered:
How does each of their performances compare to 2008 Linear model?

For the 2012 model, Votes column has been replaced with the total number of votes for 2012 although it had almost no impact on the result after testing. As the model shows below the R square and adjusted R square values are better than 2008 model. The 2012 model takes more variability into account than 2008. However it is not sure if the model is overfit and is not able to predict on the new data.<br>
To find out if the model is overfit, train-test split method  has been used. The model shows similar R square value on the test set as train set meaning the model is able to capture the genreal trend.

In [10]:
df_1['Total_votes_12'] = df_1['Republicans 12 (Votes)'] + df_1['Democrats 12 (Votes)']
df['Votes'] = df_1['Total_votes_12']

In [11]:
regression(df.iloc[:,:],df_1['Republicans12_Voteshare'])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.715
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                     180.3
Date:                Sun, 28 Apr 2019   Prob (F-statistic):               0.00
Time:                        19:56:30   Log-Likelihood:                 3468.0
No. Observations:                3141   AIC:                            -6848.
Df Residuals:                    3097   BIC:                            -6582.
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5089      4.117      0.367      0.7

In [12]:
def test_train(x,y):
    x = x.values
    X = sm.add_constant(x)
    y = y.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

    my_reg = sm.OLS(y_train, X_train).fit()

    y_train_predicted = my_reg.predict(X_train)

    y_test_predicted = my_reg.predict(X_test)

    print(sklearn.metrics.r2_score(y_train, y_train_predicted))
    print(sklearn.metrics.r2_score(y_test, y_test_predicted))

In [13]:
test_train(df.iloc[:,:],df_1['Republicans12_Voteshare'])

0.7136437963819189
0.7090432864600528


**2016 Model:**<br>
2016 model seems to consider even more variabiltiy and fit well with the data. Test-train split result shows the model is not overfit as well. The Adjusted R square has also increased. <br>

In [14]:
df_1['Total_votes_16'] = df_1['votes16_trumpd'] + df_1['votes16_clintonh']
df['Votes'] = df_1['Total_votes_16']

In [15]:
regression(df.iloc[:,:],df_1['Republicans16_Voteshare'])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     305.3
Date:                Sun, 28 Apr 2019   Prob (F-statistic):               0.00
Time:                        19:56:43   Log-Likelihood:                 3880.8
No. Observations:                3141   AIC:                            -7674.
Df Residuals:                    3097   BIC:                            -7407.
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5702      3.610     -0.435      0.6

In [16]:
test_train(df.iloc[:,:],df_1['Republicans16_Voteshare'])

0.8042515646865074
0.8165592072243069


## Comparison: 
As the comparison table shows below, the model has improved for every election year. Considering the fact that, the independent variables were not changed and every election year more counties are voting for Republic party, the variance is less in the data set every year, the model seems to reflect this information and is a good fit of the data. 

|    Metrics   	| 2008 	| 2012 	| 2016 	|
|:------------:	|:----:	|:----:	|:----:	|
|   R sqaure   	| 0.67 	| 0.71 	| 0.80 	|
|  Adj sqaure  	| 0.66 	| 0.71 	| 0.80 	|
| Prob(F-stat) 	| 0.00 	| 0.00 	| 0.00 	|

Lets see if adding the historical performance improves our model. For 2016 model, we will use the 2008 and 2012 Republicans and Democrats vote share data as independent variables. The data frame `df_test_2` was created above with these columns and will be analyzed. <br>
As shown below the model captures 97% variance and the adjusted R suqare is similar to R square. The test-train split also shows the model is not overfit. The standard error values for coefficients shows that the data points are very close to the regression line explaining better fit of the model. 

In [17]:
regression(df_test.iloc[:,:],df_1['Republicans16_Voteshare'])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.974
Method:                 Least Squares   F-statistic:                     2661.
Date:                Sun, 28 Apr 2019   Prob (F-statistic):               0.00
Time:                        19:56:54   Log-Likelihood:                 7061.0
No. Observations:                3141   AIC:                        -1.403e+04
Df Residuals:                    3095   BIC:                        -1.375e+04
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.2210      0.656     -1.861      0.0

In [18]:
test_train(df_test.iloc[:,:],df_1['Republicans16_Voteshare'])

0.9748020699359016
0.9743675091077355
